In [ ]:
import json
import pandas as pd
import geopandas as gpd
import numpy as np
import database as db
import gdal
import rasterio
from rasterstats import zonal_stats
from munging import get_centroid
from apsim.daymet import create_excel_met
from apsim.apsim_input_writer import create_mukey_runs
import matplotlib.pyplot as plt
#ogr2ogr -f "PostgreSQL" PG:"dbname=aepe port=5432 user=mjn password=KBlhsab7618 host=10.24.18.212" "accola_yield.geojson" -skip-failures

In [ ]:
dbconn = db.connect_to_db('database.ini')

In [ ]:
accola_soy_mgmt_2018 = json.loads( open( 'crop_jsons/accola_sfc_2018.json', 'r' ).read() )
accola_corn_mgmt_2019 = json.loads( open( 'crop_jsons/accola_cfs_2019.json', 'r' ).read() )

In [ ]:
accola_soils = gpd.read_postgis('SELECT * FROM biocent_farms.accola_ssurgo;', dbconn, geom_col='wkb_geometry')
accola_mukeys = list(np.unique(accola_soils['mukey']))
#accola_soils = pd.read_sql('SELECT * FROM biocent_farms.accola_ssurgo;', dbconn)
#### Get centroid and create met file
#accola_centroid = get_centroid(accola_soils, 'areasymbol', 'wkb_geometry')
#create_excel_met(accola_centroid[0], accola_centroid[1], 2012, 2019, 'accola')

In [ ]:
create_mukey_runs(accola_mukeys, dbconn, 'sfc', 'accola.met', 'Accola', sfc_mgmt=accola_soy_mgmt_2018, cfs_mgmt=accola_corn_mgmt_2019)

In [ ]:
#gdal Rasterize (vector to raster)
#raster analysis > zonal statistics
#user python rasterstats to calculate zonal statistics
# from rasterstats import zonal_stats
# stats = zonal_stats("tests/data/polygons.shp", "tests/data/elevation.tif")
# stats[1].keys()
# ['count', 'min', 'max', 'mean']
# [f['mean'] for f in stats]
# [756.6057470703125, 114.660084635416666]

In [ ]:
accola_yield = gpd.read_file("yield_monitor/Accola2019YieldwGeom.dbf")

In [ ]:
accola_yield_clean = accola_yield[['Yld_Vol_Dr','geometry']]
#accola_yield_clean['Yld_Vol_Dr'].describe()

In [ ]:
# For re-projecting input vector layer to raster projection
def reproject(vector_gpd, raster):
    proj = raster.crs.to_proj4()
    print("Original vector layer projection: ", vector_gpd.crs)
    reproj = vector_gpd.to_crs(proj)
    print("New vector layer projection (PROJ4): ", reproj.crs)
    fig, ax = plt.subplots(figsize=(15, 15))
    rplt.show(raster, ax=ax)
    reproj.plot(ax=ax, facecolor='none', edgecolor='red')
    fig.show()
    return reproj

In [ ]:
#stats list: ['min', 'max', 'mean', 'count', 'sum', 'std', 'median', 'majority', 'minority', 'unique', 'range']
def get_zonal_stats(vector, raster, stats):
    # Run zonal statistics, store result in geopandas dataframe
    result = zonal_stats(vector, raster, stats=stats, geojson_out=True)
    geostats = gpd.GeoDataFrame.from_features(result)
    return geostats

In [ ]:
# For generating raster from zonal statistics result
def stats_to_raster(zdf, raster, stats, out_raster, no_data='y'):
    meta = raster.meta.copy()
    out_shape = raster.shape
    transform = raster.transform
    dtype = raster.dtypes[0]
    field_list = list_columns(stats)
    index = int(input("Rasterize by which field? "))
    zone = zdf[field_list[index]]
    shapes = ((geom,value) for geom, value in zip(zdf.geometry, zone))
    burned = rasterize(shapes=shapes, fill=0, out_shape=out_shape, transform=transform)
    show(burned)
    meta.update(dtype=rasterio.float32, nodata=0)
    # Optional to set nodata values to min of stat
    if no_data == 'y':
        cutoff = min(zone.values)
        print("Setting nodata cutoff to: ", cutoff)
        burned[burned < cutoff] = 0 
    with rasterio.open(out_raster, 'w', **meta) as out:
        out.write_band(1, burned)
    print("Zonal Statistics Raster generated")

In [ ]:
stats = zonal_stats('yield_monitor/accola_soils.shp', 'yield_monitor/accola_2019_yield_raster.tif', geojson_out=True)

In [ ]:
gpd.GeoDataFrame.from_features(stats)

In [ ]:
#gdal.Grid('yield_avg.tif', 'accola_yield_basic.csv')

In [ ]:
len(accola_yield_clean.loc[accola_yield_clean['Yld_Vol_Dr'] <= 150])

In [ ]:
accola_mukeys = list(np.unique(accola_soils['mukey']))

In [ ]:
create_mukey_runs(accola_mukeys, dbconn, 'sfc', 'accola.met', 'Accola', sfc_mgmt=accola_soy_mgmt_2018, cfs_mgmt=accola_corn_mgmt_2019)